In [78]:
import sqlite3 
from sqlite3 import Error 
import pandas as pd

from dash import Dash, html, dcc
import plotly.express as px

Дан csv-файл с транзакциями с 2023–01-01 до 2023-04-20. Необходимо в Jupyter-ноутбуке выполнить следующие пункты, используя SQLite:

## Шаг 1

**(Балл - 0.2)** Необходимо скачать CSV-файл — «<span style="color:green">final_transactions.csv</span>», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

<br/>
(0, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - время транзакций<br/>
(1, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - клиент<br/>
(2, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакций<br/>

**Комментарий**: Можно оставить все 5 полей, но запросы будут выполняться чуть дольше.

In [79]:
def create_connection(path):
    connection = None
    try:
        conn = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return conn

In [80]:
def execute_query(query, conn):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [81]:
def table_info(table_name, conn):
    c = conn.cursor()
    meta = c.execute("PRAGMA table_info('" + table_name + "')")
    for r in meta:
        print(r)

In [82]:
conn = create_connection("SQL_final_transactions.db")

Connection to SQLite DB successful


Зададим имя таблицы в БД

In [83]:
table_name = 'transaction_bd'

In [84]:
create_table_transaction_bd = """
CREATE TABLE IF NOT EXISTS transaction_bd (
  transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
  tx_datetime NUMERIC NOT NULL,
  customer_id INTEGER NOT NULL,
  terminal_id INTEGER NOT NULL,
  tx_amount REAL NOT NULL
);
"""

In [85]:
execute_query(
    f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        tx_datetime NUMERIC NOT NULL,
        customer_id INTEGER NOT NULL,
        terminal_id INTEGER NOT NULL,
        tx_amount REAL NOT NULL
    );
    ''',
    conn)

Query executed successfully


In [86]:
trans_df = pd.read_csv('data/final_transactions.csv', delimiter=',')

In [87]:
trans_df['TX_DATETIME'] = pd.to_datetime(trans_df['TX_DATETIME']).map(pd.Timestamp.timestamp) #REAL
#trans_df['TX_DATETIME'] = pd.to_datetime(trans_df['TX_DATETIME']).astype('int64') / 10**9 #INTEGER

In [88]:
trans_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,1.672531e+09,596,3156,533.07
1,1,1.672531e+09,4961,3412,808.56
2,2,1.672532e+09,2,1365,1442.94
3,3,1.672532e+09,4128,8737,620.65
4,4,1.672532e+09,927,9906,490.66


In [89]:
trans_df.tail()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
1048570,1048570,1.681985e+09,2380,3780,325.64
1048571,1048571,1.681985e+09,738,5151,20.38
1048572,1048572,1.681985e+09,1000,5417,182.79
1048573,1048573,1.681985e+09,3028,6439,455.44
1048574,1048574,1.681985e+09,3252,6009,304.55


In [90]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  float64
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 40.0 MB


In [91]:
trans_df.to_sql('transaction_bd', conn, if_exists='replace', index=False)

1048575

In [92]:
table_info(table_name, conn)

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0)
(1, 'TX_DATETIME', 'REAL', 0, None, 0)
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0)
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0)
(4, 'TX_AMOUNT', 'REAL', 0, None, 0)


In [93]:
pd.read_sql(
    f'''
    SELECT * 
    FROM {table_name} 
    LIMIT 10
    ''', 
    conn)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,1.672531e+09,596,3156,533.07
1,1,1.672531e+09,4961,3412,808.56
2,2,1.672532e+09,2,1365,1442.94
3,3,1.672532e+09,4128,8737,620.65
4,4,1.672532e+09,927,9906,490.66
5,5,1.672532e+09,568,8803,401.17
6,6,1.672532e+09,2803,5490,938.54
7,7,1.672532e+09,4684,2486,206.53
8,8,1.672532e+09,4128,8354,253.47
9,9,1.672532e+09,541,6212,555.63


## Шаг 2

**(Балл - 0.1 за каждый пункт)** Написать следующие запросы к таблице transaction_bd:

- **A**. Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);

In [94]:
pd.read_sql(
    f'''
    SELECT customer_id 
    FROM {table_name} 
    GROUP BY customer_id 
    HAVING SUM(tx_amount) > 700000 
    ORDER BY customer_id
    ''', 
conn)

,CUSTOMER_ID
0,389
1,2249
2,2891
3,3116
4,4163


- **B**. Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);

In [95]:
pd.read_sql(
    f'''
    SELECT customer_id 
    FROM {table_name} 
    GROUP BY customer_id 
    HAVING SUM(tx_amount) > 200000 AND DATE(tx_datetime, 'unixepoch') BETWEEN '2023-01-01' AND '2023-01-13'
    ORDER BY customer_id
    ''', 
conn)

,CUSTOMER_ID
0,1
1,4
2,9
3,25
4,27
...,...
947,4974
948,4981
949,4983
950,4994


- **C**. Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;

In [96]:
pd.read_sql(
    f'''
    SELECT customer_id 
    FROM {table_name} 
    WHERE SUBSTR(customer_id, 1, 1) = '4' 
    GROUP BY customer_id 
    HAVING SUM(tx_amount) > 444
    ''', 
conn)

,CUSTOMER_ID
0,4
1,40
2,41
3,42
4,43
...,...
1095,4995
1096,4996
1097,4997
1098,4998


- **D**. Создать флаг доходности клиентов по логике:
    - **1**. Если сумма транзакций не более 50000, тогда вывести 'низкая доходность';
    - **2**. Если сумма транзакций больше 50000 и не более 100000 тогда вывести 'средняя доходность';
    - **3**. Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.

In [97]:
query_d = f'''
    SELECT customer_id, 
    CASE 
        WHEN SUM(tx_amount) <= 50000 THEN 'низкая доходность'
        WHEN SUM(tx_amount) > 50000 AND SUM(tx_amount) <= 100000 THEN 'средняя доходность'
        ELSE 'высокая доходность' 
    END customer_rate
    FROM {table_name} 
    GROUP BY CUSTOMER_ID
'''
pd.read_sql(query_d, conn)

,CUSTOMER_ID,customer_rate
0,0,высокая доходность
1,1,высокая доходность
2,2,высокая доходность
3,3,низкая доходность
4,4,высокая доходность
...,...,...
4981,4995,высокая доходность
4982,4996,низкая доходность
4983,4997,высокая доходность
4984,4998,высокая доходность


- **E**. Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);

In [98]:
pd.read_sql(
    f'''
    WITH tmp as ({query_d})
    SELECT tmp.customer_rate, count(tmp.customer_id) as quantity
    FROM tmp
    GROUP BY tmp.customer_rate
    ''', 
conn)

,customer_rate,quantity
0,высокая доходность,2167
1,низкая доходность,1812
2,средняя доходность,1007


- **F**. Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).

In [99]:
response_f = pd.read_sql(
    f'''
    SELECT DATE(tx_datetime, 'unixepoch') as date, SUM(tx_amount) as trans_sum 
    FROM {table_name}
    GROUP BY date
    ORDER BY date
    ''', 
conn)

## Шаг 3

**(Балл - 0.2)** Подготовить дашборд с помощью Dash по пункту 2.f, включив туда графики _bar_ и _histogram_; вставить в конце ноутбука скрин графиков из дашборда.

In [100]:
app = Dash()
title_1 = 'Сумма транзакций за каждый день'
title_2 = 'Распределение сумм транзакций на каждый день'

fig1 = px.bar(response_f, x="date", y="trans_sum", title = title_1)

fig2 = px.histogram(response_f, x = "trans_sum",
             title = title_2,
             nbins = 100,
             )

fig1.update_layout(template = 'plotly')
fig2.update_layout(template = 'plotly')

# публикация на веб_сервере для браузера
app.layout = html.Div(children=[

    html.H1(children = title_1),
    html.Div(children = title_2),

    dcc.Graph(
        id='example-graph',
        figure=fig1
    ),
     dcc.Graph(
        id='example-graph1',
        figure=fig2
    )
])

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [30/Mar/2023 21:39:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2023 21:39:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2023 21:39:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2023 21:39:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [30/Mar/2023 21:39:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Результат:

![](https://raw.githubusercontent.com/savspit/mipt_data_storage/main/homeworks/hw2/img/plot.png)

Закрываем подключение

In [101]:
conn.close()